## Deliverable 2. Create a Customer Travel Destinations Map.

In [37]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [38]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marystown,CA,47.1666,-55.1483,51.73,57,93,11.88,overcast clouds
1,1,Balakliya,UA,49.4627,36.8595,57.00,68,97,6.87,overcast clouds
2,2,Makakilo City,US,21.3469,-158.0858,74.19,77,100,0.00,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,43.21,98,100,3.69,light rain
4,4,Tari,NG,12.4086,4.4914,104.99,20,55,4.68,broken clouds


In [39]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What minimum temperature do you like for your trip? "))
max_temp = float(input("What maximum temperature do you like for your trip? "))

What minimum temperature do you like for your trip? 70
What maximum temperature do you like for your trip? 100


In [40]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Makakilo City,US,21.3469,-158.0858,74.19,77,100,0.00,overcast clouds
5,5,Avarua,CK,-21.2078,-159.7750,80.65,94,75,5.75,broken clouds
6,6,Bambous Virieux,MU,-20.3428,57.7575,73.65,78,40,4.61,scattered clouds
9,9,Atuona,PF,-9.8000,-139.0333,78.30,66,60,21.30,broken clouds
11,11,Vaini,TO,-21.2000,-175.2000,75.11,59,100,23.04,overcast clouds
12,12,Kapaa,US,22.0752,-159.3190,74.48,86,52,5.01,light rain
14,14,Hilo,US,19.7297,-155.0900,75.27,82,64,3.00,broken clouds
15,15,Broome,US,42.2506,-75.8330,78.19,64,55,4.00,broken clouds
16,16,Alugan,PH,12.2188,125.4808,79.79,86,93,10.27,overcast clouds
17,17,Rikitea,PF,-23.1203,-134.9692,77.61,74,53,21.50,broken clouds


In [41]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             341
City                   341
Country                340
Lat                    341
Lng                    341
Max Temp               341
Humidity               341
Cloudiness             341
Wind Speed             341
Current Description    341
dtype: int64

In [42]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

Unnamed: 0             340
City                   340
Country                340
Lat                    340
Lng                    340
Max Temp               340
Humidity               340
Cloudiness             340
Wind Speed             340
Current Description    340
dtype: int64

In [43]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Makakilo City,US,74.19,overcast clouds,21.3469,-158.0858,
5,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,
6,Bambous Virieux,MU,73.65,scattered clouds,-20.3428,57.7575,
9,Atuona,PF,78.30,broken clouds,-9.8000,-139.0333,
11,Vaini,TO,75.11,overcast clouds,-21.2000,-175.2000,
12,Kapaa,US,74.48,light rain,22.0752,-159.3190,
14,Hilo,US,75.27,broken clouds,19.7297,-155.0900,
15,Broome,US,78.19,broken clouds,42.2506,-75.8330,
16,Alugan,PH,79.79,overcast clouds,12.2188,125.4808,
17,Rikitea,PF,77.61,broken clouds,-23.1203,-134.9692,


In [44]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Makakilo City,US,74.19,overcast clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club
5,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Paradise Inn
6,Bambous Virieux,MU,73.65,scattered clouds,-20.3428,57.7575,Casa Tia Villa
9,Atuona,PF,78.30,broken clouds,-9.8000,-139.0333,Villa Enata
11,Vaini,TO,75.11,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Kapaa,US,74.48,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
14,Hilo,US,75.27,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
15,Broome,US,78.19,broken clouds,42.2506,-75.8330,Chenango Valley State Park
16,Alugan,PH,79.79,overcast clouds,12.2188,125.4808,JM Lodge
17,Rikitea,PF,77.61,broken clouds,-23.1203,-134.9692,People ThankYou


In [45]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Makakilo City,US,74.19,overcast clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club
5,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Paradise Inn
6,Bambous Virieux,MU,73.65,scattered clouds,-20.3428,57.7575,Casa Tia Villa
9,Atuona,PF,78.30,broken clouds,-9.8000,-139.0333,Villa Enata
11,Vaini,TO,75.11,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort


In [46]:
# 8a. Create the output File (CSV)
output_data_file = ('WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
current_description = hotel_df["Current Description"]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))